In [8]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer


def question_answer(question, reference):
    """
    using bert to answer some question
    """

    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

    tokens = ['[CLS]'] + tokenizer.tokenize(question) + ['[SEP]'] + tokenizer.tokenize(reference) + ['[SEP]']
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    mask = [1] * len(input_ids)
    # apply mask,0 for reference and 1 for questions
    type_ids = [0] * (1 + len(tokenizer.tokenize(question)) + 1) + [1] * (len(tokenizer.tokenize(reference)) + 1)
    input_ids, mask, type_ids = map(lambda f: tf.expand_dims(
        tf.convert_to_tensor(f, dtype=tf.int32), 0),
                                   (input_ids, mask, type_ids))

    outputs = model([input_ids, mask, type_ids])
    

    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    if short_start <= len(tokenizer.tokenize(question)) or short_start >= short_end:
        return None

    answer_tokens = tokens[short_start: short_end + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

In [9]:
with open('/kaggle/input/pld-qa/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('When are PLDs?', reference))

on - site days from 9 : 00 am to 3 : 00 pm


In [12]:
with open('/kaggle/input/pld-qa/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('What does PLD stand for?', reference))

peer learning days


In [13]:

with open('/kaggle/input/pld-qa/PeerLearningDays.md') as f:
    reference = f.read()

print(question_answer('What are Mock Interviews?', reference))

None
